In [1]:
# Cài đặt các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
# Đọc dữ liệu từ tệp CSV
review_data = pd.read_csv(r'C:\Users\anhn2\Documents\DJANGO\DA\TIKI\review.csv')

# Hiển thị thông tin dữ liệu để xác nhận
review_data.head()

,user_id,product_id,rating
0,14479838,102093936,5
1,7884174,158755500,5
2,24508580,158755500,5
3,689075,143687625,5
4,462177,143687625,5


In [3]:
# Chọn các cột cần thiết cho mô hình
r_cols = ['user_id', 'product_id', 'rating']
ratings = review_data[r_cols].values

# Chuyển đổi chỉ số từ 1-based sang 0-based (để phù hợp với ma trận)
ratings[:, :2] -= 1

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% train, 20% test)
rate_train, rate_test = train_test_split(ratings, test_size=0.2, random_state=42)

In [4]:
class MF:
    """Matrix Factorization for recommendation systems."""

    def __init__(self, Y_data, K, lam=0.1, Xinit=None, Winit=None, 
                 learning_rate=0.5, max_iter=1000, print_every=100, user_based=0):
        self.Y_raw = Y_data.copy()
        self.Y_data = Y_data.copy()
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based

        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.n_ratings = Y_data.shape[0]

        self.X = Xinit if Xinit is not None else np.random.randn(self.n_items, K)
        self.W = Winit if Winit is not None else np.random.randn(K, self.n_users)
        
        self.b = np.random.randn(self.n_items)
        self.d = np.random.randn(self.n_users)
        self.mu = 0  # Global bias
        self.muu = None

    def normalize_Y(self):
        self.Y_data = self.Y_data.astype(float) 
        if self.user_based:
            user_col, item_col, n_objects = 0, 1, self.n_users
        else:
            user_col, item_col, n_objects = 1, 0, self.n_items

        users = self.Y_data[:, user_col]
        self.muu = np.zeros((n_objects,))
        for n in range(n_objects):
            ids = np.where(users == n)[0].astype(np.int32)
            ratings = self.Y_data[ids, 2]
            mean_rating = np.mean(ratings) if len(ratings) > 0 else 0
            self.muu[n] = mean_rating
            self.Y_data[ids, 2] -= mean_rating

    def loss(self):
        L = 0
        for i in range(self.Y_data.shape[0]):
            user, item, rating = int(self.Y_data[i, 0]), int(self.Y_data[i, 1]), self.Y_data[i, 2]
            pred = self.X[item, :].dot(self.W[:, user]) + self.b[item] + self.d[user] + self.mu
            L += 0.5 * (pred - rating) ** 2
        
        L /= self.n_ratings
        L += 0.5 * self.lam * (np.linalg.norm(self.X, 'fro') + 
                               np.linalg.norm(self.W, 'fro') + 
                               np.linalg.norm(self.b) + 
                               np.linalg.norm(self.d))
        return L

    def get_items_rated_by_user(self, user_id):
        ids = np.where(self.Y_data[:, 0] == user_id)[0]
        item_ids = self.Y_data[ids, 1].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return item_ids, ratings

    def get_users_who_rate_item(self, item_id):
        ids = np.where(self.Y_data[:, 1] == item_id)[0]
        user_ids = self.Y_data[ids, 0].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return user_ids, ratings

    def updateX(self):
        for item_id in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(item_id)
            Wm = self.W[:, user_ids]
            dm = self.d[user_ids]
            error = self.X[item_id, :].dot(Wm) + self.b[item_id] + dm + self.mu - ratings
            grad_xm = error.dot(Wm.T) / self.n_ratings + self.lam * self.X[item_id, :]
            grad_bm = np.sum(error) / self.n_ratings + self.lam * self.b[item_id]
            self.X[item_id, :] -= self.learning_rate * grad_xm
            self.b[item_id] -= self.learning_rate * grad_bm

    def updateW(self):
        for user_id in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(user_id)
            Xn = self.X[item_ids, :]
            bn = self.b[item_ids]
            error = Xn.dot(self.W[:, user_id]) + bn + self.mu + self.d[user_id] - ratings
            grad_wn = Xn.T.dot(error) / self.n_ratings + self.lam * self.W[:, user_id]
            grad_dn = np.sum(error) / self.n_ratings + self.lam * self.d[user_id]
            self.W[:, user_id] -= self.learning_rate * grad_wn
            self.d[user_id] -= self.learning_rate * grad_dn

    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                print(f"Iter {it + 1}, Loss: {self.loss():.4f}, RMSE: {self.evaluate_RMSE(self.Y_raw):.4f}")

    def pred(self, user_id, item_id):
        bias = self.muu[user_id] if self.user_based else self.muu[item_id]
        pred = self.X[item_id, :].dot(self.W[:, user_id]) + self.b[item_id] + self.d[user_id] + bias
        return max(0, min(5, pred))

    def pred_for_user(self, user_id):
        rated_items = np.where(self.Y_data[:, 0] == user_id)[0]
        items_rated = self.Y_data[rated_items, 1].tolist()
        predictions = [(i, self.pred(user_id, i)) for i in range(self.n_items) if i not in items_rated]
        return predictions

    def evaluate_RMSE(self, test_data):
        errors = [
            (self.pred(int(user), int(item)) - rating) ** 2
            for user, item, rating in test_data
        ]
        return np.sqrt(np.mean(errors))


In [5]:
import joblib

# Các tham số cho K=5
K = 5
lam = 0.1
lr = 0.5

print(f"Training model with K={K}, lam={lam}, lr={lr}...")
# Khởi tạo mô hình với tham số
rs = MF(rate_train, K=K, lam=lam, learning_rate=lr, max_iter=50, print_every=10, user_based=1)
rs.fit()

# Lưu mô hình sau khi huấn luyện
model_filename = f"mf_model_K{K}_lam{lam}_lr{lr}.pkl"
joblib.dump(rs, model_filename)
print(f"Model with K={K}, lam={lam}, lr={lr} saved as {model_filename}")

# Đánh giá mô hình trên tập kiểm tra
rmse_test = rs.evaluate_RMSE(rate_test)
print(f"RMSE for K={K}, lam={lam}, lr={lr}: {rmse_test:.4f}")

Training model with K=5, lam=0.1, lr=0.5...


KeyboardInterrupt: 